In this notebook following analysis is performed:
1. Comparison of the different water treatment trains based on the brine disposal method. Select minimum, maximum and medium cost treatment trains
2. Sensitivity of the LCOW of the treatment train to the treated volume for each treatment train selected in step 1
3. Sensitivity of the LCOW of the treatment train to the distance the brine needs to be transported for disposal

In [ ]:
import pandas as pd
import numpy as np
from watertap3.utils import watertap_setup, get_case_study, run_model 
from watertap3.utils import run_watertap3, run_model_no_print, run_and_return_model
import pyomo.util.infeasible as infeas
from idaes.core.util.model_statistics import *
import matplotlib.pyplot as plt
from truck_pipe_cost_functions import truck_costing, pipe_costing
from watertap3.utils.post_processing import get_results_table

Baseline test case

In [ ]:
case_study = 'kbhdp' #'big_spring'
scenario = 'dwi' #'dwi_a'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

# m.fs.reverse_osmosis.membrane_area.unfix()
# m.fs.reverse_osmosis.feed.pressure.unfix()

# m.fs.big_spring_feed.flow_vol_in.fix(72037 * 6.30902e-5) #gpm to m3s
# m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(0.698) #kg/m3
# m.fs.well_field.lift_height.fix(174)

m.fs.ro_first_stage.membrane_area.unfix()
m.fs.ro_second_stage.membrane_area.unfix()
m.fs.ro_first_stage.feed.pressure.unfix()
m.fs.ro_second_stage.feed.pressure.unfix()

m.fs.kbhdp_brackish_ave.flow_vol_in.fix(72037 * 6.30902e-5/2) #gpm to m3s
# m.fs.kbhdp_brackish_ave.conc_mass_in[0, 'tds'].fix(0.698) #kg/m3
# m.fs.kbhdp_brackish_ave.lift_height.fix(174)

m.fs.costing_param.electricity_price = 0.0652
m = run_and_return_model(m, objective=True,print_it=True)
m, df = get_results_table(m=m, case_study='test', scenario=scenario)    
m.fs.costing.LCOW.value()

In [ ]:
(0.03577*4.54483 + 0.18993*4.54437 + 0.19989*4.31716 + 0.033*4.1543 + 0.0127*4.15388 + 0.02448*4.15347 + 0.02742* 0.40196)/4.153

In [ ]:
0.03577 + 0.18993 + 0.19989 + 0.033 + 0.0127 + 0.02448 + 0.02742

In [ ]:
m.fs.deep_well_injection.LCOW()

In [ ]:
m.fs.costing.capital_recovery_factor()

Comparison of all the brine management methods

Sensitivity of LCOW to volume of water being treated- baseline case

In [ ]:
case_study = 'big_spring'
scenario = 'dwi_a'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

m.fs.reverse_osmosis.membrane_area.unfix()
m.fs.reverse_osmosis.feed.pressure.unfix()
LCOWList_dwi_a = np.array([])
flow_list = [0.075,0.5,1.5,2.25,4.5,0.075]
for flow in flow_list:
    print(flow)
    m.fs.big_spring_feed.flow_vol_in.fix(flow)
    m = run_and_return_model(m, objective=True)
    print(m.fs.costing.LCOW.value())
    LCOWList_dwi_a= np.concatenate((LCOWList_dwi_a,[m.fs.costing.LCOW.value()]))
m, df = get_results_table(m=m, case_study='high_flow', scenario=scenario)      
LCOWList_dwi_a
dwi_a_conc_flow = m.fs.deep_well_injection.flow_vol_in[0].value
dwi_a_total_water_recovery = m.fs.costing.system_recovery()
dwi_a_update_basis = (1-dwi_a_total_water_recovery)/dwi_a_total_water_recovery

In [ ]:
4.5*dwi_a_total_water_recovery

In [ ]:
LCOWList_dwi_a

Sensitivity of LCOW to volume of water being treated-medium cost case

In [ ]:
case_study = 'big_spring'
scenario = 'zld_a'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'


m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

m.fs.ro_first_stage.membrane_area.unfix()
m.fs.ro_first_stage.feed.pressure.unfix()
m.fs.ro_second_stage.membrane_area.unfix()
m.fs.ro_second_stage.feed.pressure.unfix()

LCOWList_zld_a = np.array([])
flow_list = [0.075,0.5,1.5,2.25,4.5]
for flow in flow_list:
    print(flow)
    m.fs.big_spring_feed.flow_vol_in.fix(flow)
    # m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(0.698)
    m = run_and_return_model(m, objective=True)
    LCOWList_zld_a= np.concatenate((LCOWList_zld_a,[m.fs.costing.LCOW.value()]))
m, df = get_results_table(m=m, case_study='high_flow', scenario=scenario)     
LCOWList_zld_a
zld_a_conc_flow = m.fs.landfill_zld.flow_vol_in[0].value
zld_a_total_water_recovery = m.fs.costing.system_recovery()
zld_a_update_basis = (1-zld_a_total_water_recovery)/zld_a_total_water_recovery

Sensitivity of LCOW to volume of water being treated- highest cost case

In [ ]:
case_study = 'big_spring'
scenario = 'zld_c'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

m.fs.ro_first_stage.membrane_area.unfix()
m.fs.ro_first_stage.feed.pressure.unfix()
m.fs.ro_second_stage.membrane_area.unfix()
m.fs.ro_second_stage.feed.pressure.unfix()

LCOWList_zld_c = np.array([])
flow_list = [4.5,0.5,1.5,2.25,4.5]
for flow in flow_list:
    print(flow)
    m.fs.big_spring_feed.flow_vol_in.fix(flow)
    m = run_and_return_model(m, objective=True,print_it=True)
    LCOWList_zld_c= np.concatenate((LCOWList_zld_c,[m.fs.costing.LCOW.value()]))
m, df = get_results_table(m=m, case_study='high_flow', scenario=scenario)     
LCOWList_zld_c
zld_c_conc_flow = m.fs.landfill_zld.flow_vol_in[0].value
zld_c_total_water_recovery = m.fs.costing.system_recovery()
zld_c_update_basis = (1-zld_c_total_water_recovery)/zld_c_total_water_recovery

In [ ]:
zld_a_total_water_recovery

Plot LCOW sensitivity to flow for each treatment train train

In [ ]:
fig, ax = plt.subplots(figsize = (7,7))

ax.plot(flow_list,LCOWList_dwi_a, label = 'Deep Well Injection',linewidth = 3)
ax.plot(flow_list,LCOWList_zld_a, label = 'Evaporator Pond',linewidth = 3)
ax.plot(flow_list,LCOWList_zld_c, label = 'Brine Concentrator + Crystallizer',linewidth = 3)

ax.set_ylabel(r'LCOW (\$/m${^3}$)', fontsize = 20)
ax.set_xlabel(r'Volumetric Flow Rate (m${^3}$/s)', fontsize = 20)
ax.tick_params(axis='both', which='major', labelsize=20)

ax.set_ylim([0,2])
ax.set_xlim([0,5])

ax.legend(fontsize = 16, frameon = False)

Transport cost as a function of mode, distance and volume

In [ ]:
wacc = 0.1
plant_lifetime_yrs = 30

capital_recovery_factor = (wacc * (1 + wacc) ** plant_lifetime_yrs) / (((1 + wacc) ** plant_lifetime_yrs) - 1)
capital_recovery_factor

Sensitivity check on deep well injection case with transport distance being varied:
1. This is the baseline cost case for the AMO project- verify this?
2. Default pipe transport cost is used

In [ ]:
# Volume of concentrate in m3/s for input treatment volume 4.5 m3/s
print(dwi_a_conc_flow)
print(zld_a_conc_flow)
print(zld_c_conc_flow)

In [ ]:
# Calculate truck transport cost for alls cases
transport_costs = pd.DataFrame(columns=['process','capacity','distance',
                                        'truck_cost ($/m3 brine)','pipe_cost ($/m3 brine)',
                                        'truck_tLCOW ($/m3 brine)','pipe_tLCOW ($/m3 brine)'])
# print(transport_costs)
distances = [0,10,20,50,100] # in km
capacity = {'dwi_a':dwi_a_conc_flow*3600*24,
            'zld_a':zld_a_conc_flow*3600*24,
            'zld_c':zld_c_conc_flow*3600*24}  # m3/day

LCOW = {'dwi_a':LCOWList_dwi_a[-1],
        'zld_a':LCOWList_zld_a[-1],
        'zld_c':LCOWList_zld_c[-1]}

total_water_recovery = {'dwi_a':dwi_a_total_water_recovery,
                        'zld_a':zld_a_total_water_recovery,
                        'zld_c':zld_c_total_water_recovery}

for case in ['dwi_a','zld_a','zld_c']:
    for d in distances:
        temp = {'process':case,
                'capacity': capacity[case],
                'distance':d,
                'truck_cost ($/m3 brine)': truck_costing(d),
                'pipe_cost ($/m3 brine)':pipe_costing(capacity[case],d),
                'truck_tLCOW ($/m3 brine)': LCOW[case]+truck_costing(d),
                'pipe_tLCOW ($/m3 brine)':LCOW[case]+pipe_costing(capacity[case],d)}
        # # print(temp)
        transport_costs=transport_costs.append(temp,ignore_index=True)

print(transport_costs)

Treatment sensitivity to transport plots

In [ ]:
# Deep well injection
# Getting each unit process for treatment train
df = pd.read_csv('/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/results/case_studies/high_flow_dwi_a.csv')
units = df['Unit Process Name'].unique() 
units = np.append(units,'Distance')

data_dwi_a = pd.DataFrame( columns = units[1::])

for d in distances:
    new_row = [df[(df['Variable'] == 'Unit LCOW') ]['Value'].values]
    new_row= np.append(new_row,d)
    new_row = new_row.reshape((1,8))

    temp = pd.DataFrame(new_row , columns = units[1::],index = ['dwi_a'] )
    data_dwi_a= pd.concat([data_dwi_a,temp])

data_dwi_a.reset_index(inplace=True)
data_dwi_a = data_dwi_a.rename(columns={'index':'scenario'})
print(data_dwi_a)


In [ ]:
print(dwi_a_update_basis)
print(zld_a_update_basis)
print(zld_c_update_basis)

In [ ]:
temp=[]
for d in distances:
    temp_tranport = (transport_costs.loc[transport_costs['process']=='dwi_a'].loc[transport_costs['distance']== d]['pipe_cost ($/m3 brine)'].values[0])*dwi_a_update_basis
    temp.append(temp_tranport)
    

data_dwi_a['Transport']=temp
data_dwi_a

In [ ]:
ax = data_dwi_a[[ 'Well Field', 'Microfiltration', 'Reverse Osmosis', 'UV/AOP',
 'Treated Storage', 'Municipal Drinking' ,'Deep Well Injection' ,'Transport','Distance']].plot.bar(x='Distance',stacked = True)
plt.figure(figsize=(10,7))
ax.set_ylim([0,2])
ax.set_xticklabels([0,10,20,50,100], rotation = 0,fontsize =14)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Distance (km)',fontsize =20)
ax.set_ylabel(r'LCOW (\$/m${^3}$)',fontsize =20)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize=16,frameon=False)
ax.set_title('Deep Well Injection',fontsize =20)


In [ ]:
# ZLD a
# Getting each unit process for treatment train
df = pd.read_csv('/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/results/case_studies/high_flow_zld_a.csv')
# df = pd.read_csv('/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/results/case_studies/big_spring_zld_a.csv')
units = df['Unit Process Name'].unique() 
units = np.append(units,'Distance')

data_zld_a = pd.DataFrame( columns = units[1::])

for d in distances:
    new_row = [df[(df['Variable'] == 'Unit LCOW') ]['Value'].values]
    new_row= np.append(new_row,d)
    new_row = new_row.reshape((1,14))

    temp = pd.DataFrame(new_row , columns = units[1::],index = ['zld_a'] )
    data_zld_a= pd.concat([data_zld_a,temp])

data_zld_a.reset_index(inplace=True)
data_zld_a = data_zld_a.rename(columns={'index':'scenario'})
print(data_zld_a)


In [ ]:
temp=[]
for d in distances:
    temp_tranport =(transport_costs.loc[transport_costs['process']=='zld_a'].loc[transport_costs['distance']== d]['truck_cost ($/m3 brine)'].values[0])*(zld_a_update_basis)
    temp.append(temp_tranport)

data_zld_a['Transport (Truck)']=temp
data_zld_a

In [ ]:
ax = data_zld_a[[ 'Well Field', 'Microfiltration', 'RO First Stage',
       'Passthrough', 'Ozone', 'Bio Active Filtration', 'Lime Softening',
       'RO Second Stage', 'UV/AOP', 'Treated Storage', 'Evaporation Pond',
       'Municipal Drinking', 'Landfill ZLD', 'Transport (Truck)','Distance']].plot.bar(x='Distance',stacked = True)
plt.figure(figsize=(10,7))
ax.set_ylim([0,2])
ax.set_xticklabels([0,10,20,50,100], rotation = 0,fontsize =20)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Distance (km)',fontsize =20)
ax.set_ylabel(r'LCOW (\$/m${^3}$)',fontsize = 20)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize=16, frameon=False)
ax.set_title('Evaporation Pond + Landfill\n(MLD)',fontsize=20)

In [ ]:
LCOWList_zld_a[-1]+0.382709

In [ ]:
# ZLD c
# Getting each unit process for treatment train
df = pd.read_csv('/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/results/case_studies/high_flow_zld_c.csv')
units = df['Unit Process Name'].unique() 
units = np.append(units,'Distance')

data_zld_c = pd.DataFrame( columns = units[1::])

for d in distances:
    new_row = [df[(df['Variable'] == 'Unit LCOW') ]['Value'].values]
    new_row= np.append(new_row,d)
    new_row = new_row.reshape((1,15))

    temp = pd.DataFrame(new_row , columns = units[1::],index = ['zld_c'] )
    data_zld_c= pd.concat([data_zld_c,temp])

data_zld_c.reset_index(inplace=True)
data_zld_c = data_zld_c.rename(columns={'index':'scenario'})
print(data_zld_c)


In [ ]:
temp=[]
for d in distances:
    temp_tranport = (transport_costs.loc[transport_costs['process']=='zld_c'].loc[transport_costs['distance']== d]['truck_cost ($/m3 brine)'].values[0])*((zld_c_update_basis))
    temp.append(temp_tranport)

data_zld_c['Transport (Truck)']=temp

data_zld_c

In [ ]:
LCOWList_zld_c[-1]#+0.286637

In [ ]:
units

In [ ]:
ax = data_zld_c[[ 'Well Field', 'Microfiltration', 'RO First Stage',
       'Passthrough', 'Ozone', 'Bio Active Filtration', 'Lime Softening',
       'RO Second Stage', 'UV/AOP', 'Treated Storage', 'Brine Concentrator', 'Crystallizer', 'Municipal Drinking',
       'Landfill ZLD', 'Transport (Truck)','Distance']].plot.bar(x='Distance',stacked = True)
plt.figure(figsize=(10,7))

ax.set_ylim([0,2])
ax.set_xticklabels([0,10,20,50,100], rotation = 0,fontsize =20)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Distance (km)',fontsize = 20)
ax.set_ylabel(r'LCOW (\$/m${^3}$)',fontsize = 20)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize = 16, frameon= False)
ax.set_title('Brine Concentrator + Crystallizer + \nLandfill (ZLD)',fontsize =20)

Old method here onwards. Ignore!

In [ ]:
# case_study = 'big_spring'
# scenarios = ['dwi_a_0','dwi_a_10', 'dwi_a_20','dwi_a_50','dwi_a_100']
# desired_recovery = 1
# ro_bounds = 'other' # or 'seawater'

# LCOWList = np.array([])

# for scenario in scenarios:
#     m = watertap_setup(case_study=case_study, scenario=scenario, treatment_file_path = 'data/treatment_train_big_spring_dwi_a_sensitivity.csv')
#     m = get_case_study(m=m)
#     m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds, treatment_file_path = 'data/treatment_train_big_spring_dwi_a_sensitivity.csv')
#     m.fs.reverse_osmosis.membrane_area.unfix()
#     m.fs.reverse_osmosis.feed.pressure.unfix()
#     m = run_and_return_model(m, objective=True)
#     LCOWList= np.concatenate((LCOWList,[scenario,m.fs.costing.LCOW.value()]))

# LCOWList

Plotting the contribution of unit process LCOW to system LCOW as a function of distance from brine disposal site

In [ ]:
# scenarios = ['dwi_a_0','dwi_a_10', 'dwi_a_20','dwi_a_50','dwi_a_100']

# df = pd.read_csv('/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/results/case_studies/big_spring_dwi_a_10.csv')
# units = df['Unit Process Name'].unique() 

# data = pd.DataFrame( columns = units[1::])

# for scenario in scenarios:
#     df = pd.read_csv('/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/results/case_studies/big_spring_' + scenario + '.csv')
#     new_row = [df[(df['Variable'] == 'Unit LCOW') ]['Value'].values]
#     temp = pd.DataFrame(new_row , columns = units[1::],index= [scenario] )

#     data = pd.concat([data,temp])

# data.reset_index(inplace=True)
# data = data.rename(columns={'index':'scenario'})
# print(data)
# ax = data.plot.bar(x='scenario',stacked = True)
# ax.set_ylim([0,2.5])
# ax.set_xticklabels([0,10,20,50,100], rotation = 0,fontsize =14)
# ax.tick_params(axis='both', which='major', labelsize=14)
# ax.set_xlabel('Distance (km)',fontsize =14)
# ax.set_ylabel('LCOW (\$/${m^3}$)',fontsize =14)
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
# # data.sum()
# perc_tranport = data['Deep Well Injection']/data.sum(axis=1)*100
# perc_tranport = ["{:.2f}".format(x)+"%" for x in perc_tranport ]
# ax.bar_label(ax.containers[-1],labels=perc_tranport,label_type ='edge',fontsize =10)
# ax.set_title('Deep Well Injection',fontsize =14)

Sensitivity of the baseline condition to flow. This is to verify treating the entire volume of well yield makes sense.

In [ ]:
# # Using the maximum and minimum well yield

# max_well_yield = 4.5448 #m3/s
# min_well_yield = 0.010094 * 6.30902e-5 #m3/s
# max_LCOW_list = []
# min_LCOW_list = []

# scenarios = ['25%','50%','100%','150%','200%']
# i=0
# for well_yield in [0.25*max_well_yield,0.5*max_well_yield,max_well_yield,1.5*max_well_yield,2*max_well_yield]:
#     print(scenarios[i])
#     m = watertap_setup(case_study=case_study, scenario='dwi_a')
#     m = get_case_study(m=m)
#     m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

#     m.fs.reverse_osmosis.membrane_area.unfix()
#     m.fs.reverse_osmosis.feed.pressure.unfix()
    
#     m.fs.big_spring_feed.flow_vol_in.fix(well_yield)
#     m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(0.698)
#     m.fs.well_field.lift_height.fix(174)

#     m.fs.costing_param.electricity_price = 0.0652
#     m = run_and_return_model(m, objective=True)
#     max_LCOW_list= np.concatenate((max_LCOW_list,[scenarios[i],m.fs.costing.LCOW.value()]))
#     i=i+1

# print(max_LCOW_list)


Sensitivity check on evaporation pond and zld case with transport distance being varied:
1. This is the midlevel cost case for the AMO project- verify this?
2. Need to add transportation cost for solid transport?

In [ ]:
# case_study = 'big_spring'
# # scenario = 'baseline'
# scenarios = ['zld_a_100']#,'zld_a_10', 'zld_a_20','zld_a_50','zld_a_100']
# desired_recovery = 1
# ro_bounds = 'other' # or 'seawater'

# LCOWList = np.array([])

# for scenario in scenarios:
#     m = watertap_setup(case_study=case_study, scenario=scenario,
#                        treatment_file_path = 'data/treatment_train_big_spring_zld_a_sensitivity.csv')
#     m = get_case_study(m=m)
#     m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds,
#                       treatment_file_path = 'data/treatment_train_big_spring_zld_a_sensitivity.csv')
#     m.fs.ro_first_stage.membrane_area.unfix()
#     m.fs.ro_first_stage.feed.pressure.unfix()
#     m.fs.ro_second_stage.membrane_area.unfix()
#     m.fs.ro_second_stage.feed.pressure.unfix()
#     m = run_and_return_model(m, objective=True)
#     LCOWList= np.concatenate((LCOWList,[scenario,m.fs.costing.LCOW.value()]))

# LCOWList

In [ ]:
# scenarios = ['zld_a_0','zld_a_10', 'zld_a_20','zld_a_50','zld_a_100']

# df = pd.read_csv('/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/results/case_studies/big_spring_zld_a_10.csv')
# units = df['Unit Process Name'].unique() 

# data = pd.DataFrame( columns = units[1::])

# for scenario in scenarios:
#     df = pd.read_csv('/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/results/case_studies/big_spring_' + scenario + '.csv')
#     new_row = [df[(df['Variable'] == 'Unit LCOW') ]['Value'].values]
#     temp = pd.DataFrame(new_row , columns = units[1::],index= [scenario] )

#     data = pd.concat([data,temp])

# data.reset_index(inplace=True)
# data = data.rename(columns={'index':'scenario'})
# print(data)
# ax = data.plot.bar(x='scenario',stacked = True)
# ax.set_ylim([0,2.5])
# ax.set_xticklabels([0,10,20,50,100], rotation = 0,fontsize =14)
# ax.tick_params(axis='both', which='major', labelsize=14)
# ax.set_xlabel('Distance (km)',fontsize =14)
# ax.set_ylabel('LCOW (\$/${m^3}$)',fontsize =14)
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# # data.sum()
# perc_tranport = data['Landfill ZLD']/data.sum(axis=1)*100
# perc_tranport = ["{:.2f}".format(x)+"%" for x in perc_tranport ]
# ax.bar_label(ax.containers[-1],labels=perc_tranport,label_type ='edge',fontsize=10)
# ax.set_title('Evaporation Pond + Landfill',fontsize=14)



Sensitivity check on crystallizer and zld case with transport distance being varied:
1. This is the maximum cost case for the AMO project- verify this?
2. Need to add transportation cost for solid transport?

In [ ]:
# case_study = 'big_spring'
# # scenario = 'baseline'
# scenarios = ['zld_c_0','zld_c_10', 'zld_c_20','zld_c_50','zld_c_100']
# desired_recovery = 1
# ro_bounds = 'other' # or 'seawater'

# LCOWList = np.array([])

# for scenario in scenarios:
#     m = watertap_setup(case_study=case_study, scenario=scenario, treatment_file_path = 'data/treatment_train_big_spring_zld_c_sensitivity.csv')
#     m = get_case_study(m=m)
#     m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds, treatment_file_path = 'data/treatment_train_big_spring_zld_c_sensitivity.csv')
#     m.fs.ro_first_stage.membrane_area.unfix()
#     m.fs.ro_first_stage.feed.pressure.unfix()
#     m.fs.ro_second_stage.membrane_area.unfix()
#     m.fs.ro_second_stage.feed.pressure.unfix()
#     m = run_and_return_model(m, objective=True)    
#     LCOWList= np.concatenate((LCOWList,[scenario,m.fs.costing.LCOW.value()]))

# LCOWList

In [ ]:
# scenarios = ['zld_c_0','zld_c_10', 'zld_c_20','zld_c_50','zld_c_100']
# df = pd.read_csv('/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/results/case_studies/big_spring_zld_c_10.csv')
# units = df['Unit Process Name'].unique() 

# data = pd.DataFrame( columns = units[1::])

# for scenario in scenarios:
#     df = pd.read_csv('/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/results/case_studies/big_spring_' + scenario + '.csv')
#     new_row = [df[(df['Variable'] == 'Unit LCOW') ]['Value'].values]
#     temp = pd.DataFrame(new_row , columns = units[1::],index= [scenario] )

#     data = pd.concat([data,temp])

# data.reset_index(inplace=True)
# data = data.rename(columns={'index':'scenario'})
# # print(data)

# # print(data.sum(axis=1))
# # print(data['Landfill ZLD']/data.sum(axis=1)*100)


# ax = data.plot.bar(x='scenario',stacked = True)
# ax.set_ylim([0,2.5])
# ax.set_xticklabels([0,10,20,50,100], rotation = 0,fontsize =14)
# ax.tick_params(axis='both', which='major', labelsize=14)
# ax.set_xlabel('Distance (km)',fontsize =14)
# ax.set_ylabel('LCOW (\$/${m^3}$)',fontsize =14)
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# perc_tranport = data['Landfill ZLD']/data.sum(axis=1)*100

# perc_tranport = ["{:.2f}".format(x)+"%" for x in perc_tranport ]
# ax.bar_label(ax.containers[-1],labels=perc_tranport,label_type ='edge',fontsize =10)
# ax.set_title('Brine Concentrator + Crystallizer + Landfill',fontsize =14)
# # data.sum()

    